## Imports

In [5]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np

import unets as un
import tf_utils as tfu

## Instantiate the network

In [4]:
# Instantiate the model
unet = un.UNet4(input_size=(256, 256, 3), num_classes=1)

# Call the model with a random input to build the model
out = unet.call(tf.random.normal((1, 256, 256, 3)))

## Load only weights

In [8]:
# Instantiate the model
unet = un.UNet4cSE(input_size=(256, 256, 3), num_classes=1)

# Call the model with a random input to build the model
out = unet.call(tf.random.normal((1, 256, 256, 3)))

unet.build((1, 256, 256, 3))

# Restore the weights
unet.load_weights("Models//unet_cse_aug_4.weights.h5")


# Testing

## Preparing Testing set

In [9]:
# Set the directory paths
test_image_folder = 'ISIC-Merged\\test_images'
test_mask_folder = 'ISIC-Merged\\test_masks'

# Load the dataset
test_dataset = tfu.load_data(test_image_folder, test_mask_folder)

# Check the dataset shapes (optional)
for images, masks in test_dataset.take(1):
    print("Image batch shape:", images.shape)
    print("Mask batch shape:", masks.shape)

Image batch shape: (16, 256, 256, 3)
Mask batch shape: (16, 256, 256, 1)


## Compile and Evaluate

In [10]:
# Compile the model
unet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tfu.bce_dice_loss, metrics=[tfu.jaccard_index])

In [ ]:
# Evaluate the model on the test dataset
loss, accuracy = unet.evaluate(test_dataset)

# Visualizations

In [ ]:
test_images_dir = '/kaggle/input/isic-2016-2017/split/test_images'
test_masks_dir = '/kaggle/input/isic-2016-2017/split/test_masks'

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Get sorted image filenames (only images, not masks)
image_filenames = sorted([f for f in os.listdir(test_images_dir) if f.endswith(".jpg")])

# Iterate Over Images and Select Corresponding Mask
image_list = []
y_true_list = []
y_pred_list = []

for img_filename in image_filenames:
    img_path = os.path.join(test_images_dir, img_filename)

    # Construct the corresponding mask filename
    mask_filename = img_filename.replace(".jpg", "_segmentation.png")
    mask_path = os.path.join(test_masks_dir, mask_filename)

    # Load image and mask using your existing functions
    image_np = load_image(img_path).numpy()
    mask_tensor = load_mask(mask_path)  # Full mask path is passed correctly

    if mask_tensor is None:
        print(f" Skipping: No mask found for {img_filename}")
        continue  # Skip this image if the mask is missing

    mask_np = mask_tensor.numpy()

    # Get model prediction
    predicted_mask = unet.predict(np.expand_dims(image_np, axis=0), verbose=0)  # Add batch dimension

    # Threshold prediction to binary mask
    predicted_mask = (predicted_mask > 0.5).astype(np.uint8)

    # Store correctly paired results
    image_list.append(image_np)
    y_true_list.append(mask_np)
    y_pred_list.append(predicted_mask.squeeze())  # Remove batch dimension

# Convert Lists to Arrays
image_data = np.array(image_list)  # Shape: (N, 256, 256, 3)
y_true = np.array(y_true_list)  # Shape: (N, 256, 256, 1)
y_pred = np.array(y_pred_list)  # Shape: (N, 256, 256, 1)


In [ ]:
# Visualize First Image-Mask-Prediction Pairs
num_images_to_show = 20
fig, axes = plt.subplots(num_images_to_show, 3, figsize=(20, 20))

for i in range(num_images_to_show):
    axes[i, 0].imshow(image_data[i])
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(np.squeeze(y_true[i]), cmap="gray")
    axes[i, 1].set_title("Ground Truth Mask")
    axes[i, 1].axis("off")

    axes[i, 2].imshow(np.squeeze(y_pred[i]), cmap="gray")
    axes[i, 2].set_title("Predicted Mask")
    axes[i, 2].axis("off")

plt.tight_layout()
plt.show()

## Prediction on custom images

In [ ]:
custom_images_dir = '/kaggle/input/my-skin-lesions/my_normalized_skin_lesions'

# Get sorted image filenames (only images, not masks)
image_filenames = sorted([f for f in os.listdir(custom_images_dir) if f.endswith(".jpg")])

# Iterate Over Images and Select Corresponding Mask
image_list = []
y_pred_list = []

for img_filename in image_filenames:
    img_path = os.path.join(custom_images_dir, img_filename)

    # Load image and mask using your existing functions
    image_np = load_image(img_path).numpy()

    # Get model prediction
    predicted_mask = unet.predict(np.expand_dims(image_np, axis=0))  # Add batch dimension

    # Threshold prediction to binary mask
    predicted_mask = (predicted_mask > 0.5).astype(np.uint8)

    # Store correctly paired results
    image_list.append(image_np)
    y_pred_list.append(predicted_mask.squeeze())  # Remove batch dimension

# Convert Lists to Arrays
image_data = np.array(image_list)  # Shape: (N, 256, 256, 3)
y_pred = np.array(y_pred_list)  # Shape: (N, 256, 256, 1)


In [ ]:
# Visualize First Image-Mask-Prediction Pairs
num_images_to_show = 3
fig, axes = plt.subplots(num_images_to_show, 2, figsize=(5, 5))

for i in range(num_images_to_show):
    axes[i, 0].imshow(image_data[i])
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(np.squeeze(y_pred[i]), cmap="gray")
    axes[i, 1].set_title("Predicted Mask")
    axes[i, 1].axis("off")

plt.tight_layout()
plt.show()